In [23]:
# Split - US and Mexico

In [24]:
import pandas as pd


df = pd.read_csv('Border_Crossing_Entry_Data.csv')

ca = df[df['Border'] == 'US-Canada Border']

In [25]:
# Split - US and Canada
# Convert to DataFrame
# Adding 'Year' and 'Season' columns
ca['Date'] = pd.to_datetime(ca['Date'], format='%b %Y')
ca['Year'] = ca['Date'].dt.year
ca['Month'] = ca['Date'].dt.month

def month_to_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    elif month in [9, 10, 11]:
        return 'Fall'

ca['Season'] = ca['Month'].apply(month_to_season)


<ipython-input-25-a1adc5c00ac3>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ca['Date'] = pd.to_datetime(ca['Date'], format='%b %Y')
<ipython-input-25-a1adc5c00ac3>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ca['Year'] = ca['Date'].dt.year
<ipython-input-25-a1adc5c00ac3>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

In [26]:
# Splitting time
df

,Port Name,State,Port Code,Border,Date,Measure,Value,Latitude,Longitude,Point
0,Roma,Texas,2310.0,US-Mexico Border,Dec 2023,Buses,46.0,26.404,-99.019,POINT (-99.018981 26.403928)
1,Del Rio,Texas,2302.0,US-Mexico Border,Dec 2023,Trucks,6552.0,29.327,-100.928,POINT (-100.927612 29.326784)
2,Roma,Texas,2310.0,US-Mexico Border,Nov 2023,Trucks,3753.0,26.404,-99.019,POINT (-99.018981 26.403928)
3,Douglas,Arizona,2601.0,US-Mexico Border,Oct 2023,Buses,13.0,31.334,-109.560,POINT (-109.560344 31.334043)
4,Beecher Falls,Vermont,206.0,US-Canada Border,Aug 2023,Trucks,422.0,45.013,-71.505,POINT (-71.505309 45.013411)
...,...,...,...,...,...,...,...,...,...,...
17873,Porthill,Idaho,3308.0,US-Canada Border,Aug 2021,Truck Containers Loaded,7.0,49.000,-116.499,POINT (-116.49925 48.999861)
17874,Eagle Pass,Texas,2303.0,US-Mexico Border,Jan 2021,Truck Containers Loaded,9090.0,28.705,-100.512,POINT (-100.511977 28.705483)
17875,Ogdensburg,New York,701.0,US-Canada Border,Oct 2023,Personal Vehicles,21699.0,44.733,-75.458,POINT (-75.4577501759 44.7330898624)
17876,Warroad,Minnesota,3423.0,US-Canada Border,Oct 2023,Personal Vehicle Passengers,11722.0,48.999,-95.377,POINT (-95.376555 48.999)


In [27]:
# Calculate the average measure per year for each port
avg_measure_per_year = ca.groupby(['Port Name', 'Year'])['Value'].mean().reset_index()

# Rename columns for clarity
avg_measure_per_year.rename(columns={'Value': 'Average Measure'}, inplace=True)

# Print the resulting DataFrame
print(avg_measure_per_year)


        Port Name  Year  Average Measure
0           Alcan  1996       638.000000
1           Alcan  1997       270.500000
2           Alcan  1998        18.000000
3           Alcan  2000         0.000000
4           Alcan  2003         0.000000
..            ...   ...              ...
652     Wildhorse  2023       569.733333
653  Willow Creek  2020        27.000000
654  Willow Creek  2021        33.454545
655  Willow Creek  2022       241.866667
656  Willow Creek  2023       403.533333

[657 rows x 3 columns]


In [28]:
# Group by Border and Measure to get the average count of each vehicle type
average_measure_per_border_ca = ca.groupby(['Border', 'Measure', 'Year'])['Value'].mean().reset_index()
print(average_measure_per_border_ca)


               Border         Measure  Year        Value
0    US-Canada Border  Bus Passengers  1996   638.000000
1    US-Canada Border  Bus Passengers  1997    21.500000
2    US-Canada Border  Bus Passengers  1999  3293.000000
3    US-Canada Border  Bus Passengers  2001  2393.000000
4    US-Canada Border  Bus Passengers  2002  1586.000000
..                ...             ...   ...          ...
179  US-Canada Border          Trucks  2019  4859.250000
180  US-Canada Border          Trucks  2020  5778.672727
181  US-Canada Border          Trucks  2021  4858.044527
182  US-Canada Border          Trucks  2022  6177.031068
183  US-Canada Border          Trucks  2023  6095.351923

[184 rows x 4 columns]


In [29]:
# Calculate density (assuming 'Area' column is in the dataset, or you can sum values if no Area is provided)
ca['Density'] = ca['Value'] / ca['Value'].sum()

# Rank by Density
ranked_density_ca = ca.groupby('Port Name')['Density'].sum().sort_values(ascending=False).reset_index()
print(ranked_density_ca)

# Example visualization with Folium
import folium

map_ca = folium.Map(location=[56.1304, -106.3468], zoom_start=4)
for index, row in ranked_density_ca.iterrows():
    port_data = ca[ca['Port Name'] == row['Port Name']].iloc[0]
    folium.Marker([port_data['Latitude'], port_data['Longitude']],
                  popup=f"Port: {row['Port Name']}, Density: {row['Density']}").add_to(map_ca)
map_ca.save('ca_density_map.html')


<ipython-input-29-f6ffc28b9adf>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ca['Density'] = ca['Value'] / ca['Value'].sum()


                 Port Name   Density
0    Buffalo Niagara Falls  0.200035
1                  Detroit  0.176554
2                   Blaine  0.123577
3               Port Huron  0.075098
4   Champlain Rouses Point  0.047745
..                     ...       ...
80                 Ambrose  0.000025
81                Whitlash  0.000023
82                  Hannah  0.000013
83               Ketchikan  0.000006
84                   Noyes  0.000005

[85 rows x 2 columns]


In [30]:
import pandas as pd
import folium

# Load the data
df = pd.read_csv('Border_Crossing_Entry_Data.csv')

# Add 'Year' and 'Season' columns
df['Date'] = pd.to_datetime(df['Date'], format='%b %Y')
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month

def month_to_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    elif month in [9, 10, 11]:
        return 'Fall'

df['Season'] = df['Month'].apply(month_to_season)

# Add a column for the border type
df['Border Type'] = df['Border'].apply(lambda x: 'US-Canada' if 'Canada' in x else 'US-Mexico')

# Calculate total value per season and border type
total_value_per_season_border = df.groupby(['Season', 'Year', 'Border Type'])['Value'].sum().reset_index()
total_value_per_season_border.rename(columns={'Value': 'Total Value'}, inplace=True)

# Merge with the original data to get the total value for each season and border type
df = pd.merge(df, total_value_per_season_border, on=['Season', 'Year', 'Border Type'])

# Calculate density for each port, season, and border type
df['Density'] = df['Value'] / df['Value'].sum()

# Group by Port Name, Season, Year, Measure, and Border Type to calculate average density
avg_density_per_measure_season_border = df.groupby(['Port Name', 'Season', 'Year', 'Measure', 'Border Type'])['Density'].mean().reset_index()

# Rank the ports by density within each measure, season, year, and border type
avg_density_per_measure_season_border['Rank'] = avg_density_per_measure_season_border.groupby(['Season', 'Year', 'Measure', 'Border Type'])['Density'].rank(ascending=False)

# Sort the DataFrame by Season, Year, Measure, Border Type, and Rank
ranked_density_per_measure_season_border = avg_density_per_measure_season_border.sort_values(by=['Season', 'Year', 'Measure', 'Border Type', 'Rank']).reset_index(drop=True)

# Filter the highest value for each measure in each season, year, and border type
highest_value_measure_dfs_season_border = ranked_density_per_measure_season_border[ranked_density_per_measure_season_border['Rank'] == 1]

# Display the combined DataFrame
print(highest_value_measure_dfs_season_border)

# Save the DataFrame as a CSV file
highest_value_measure_dfs_season_border.to_csv('highest_rank.csv', index=False)


      Port Name  Season  Year                  Measure Border Type  \
0        Noonan    Fall  1996                    Buses   US-Canada   
1      Eastport    Fall  1996        Personal Vehicles   US-Canada   
3        Scobey    Fall  1996                   Trains   US-Canada   
4        Tecate    Fall  1996                   Trains   US-Mexico   
5      Calexico    Fall  1996                   Trucks   US-Mexico   
...         ...     ...   ...                      ...         ...   
15693    Laredo  Winter  2023   Truck Containers Empty   US-Mexico   
15714   Detroit  Winter  2023  Truck Containers Loaded   US-Canada   
15785    Laredo  Winter  2023  Truck Containers Loaded   US-Mexico   
15807   Detroit  Winter  2023                   Trucks   US-Canada   
15882    Laredo  Winter  2023                   Trucks   US-Mexico   

            Density  Rank  
0      0.000000e+00   1.0  
1      3.265056e-06   1.0  
3      0.000000e+00   1.0  
4      2.607872e-09   1.0  
5      1.248214e-05

In [31]:
# Ranking each Measure of transportation by year

# Group by Port Name, Year, and Measure to calculate average density
avg_density_per_measure = ca.groupby(['Port Name', 'Year', 'Measure'])['Density'].mean().reset_index()

# Rank the ports by density within each measure per year
avg_density_per_measure['Rank'] = avg_density_per_measure.groupby(['Year', 'Measure'])['Density'].rank(ascending=False)

# Sort the DataFrame by Year, Measure, and Rank
ranked_density_per_measure = avg_density_per_measure.sort_values(by=['Year', 'Measure', 'Rank']).reset_index(drop=True)

# Create a dictionary to hold DataFrames for each measure
measure_dfs = {}
for measure in ranked_density_per_measure['Measure'].unique():
    measure_dfs[measure] = ranked_density_per_measure[ranked_density_per_measure['Measure'] == measure]
measure_dfs


{'Bus Passengers':         Port Name  Year         Measure       Density  Rank
 0           Alcan  1996  Bus Passengers  6.536239e-06   1.0
 15      Fort Kent  1997  Bus Passengers  4.405302e-07   1.0
 16         Morgan  1997  Bus Passengers  0.000000e+00   2.0
 48          Sumas  1999  Bus Passengers  3.373642e-05   1.0
 75         Calais  2001  Bus Passengers  2.451602e-05   1.0
 ...           ...   ...             ...           ...   ...
 2153      St John  2023  Bus Passengers  2.151427e-07  49.0
 2154       Turner  2023  Bus Passengers  1.844080e-07  50.0
 2155  Bridgewater  2023  Bus Passengers  4.097956e-08  51.0
 2156    Limestone  2023  Bus Passengers  1.024489e-08  52.5
 2157       Opheim  2023  Bus Passengers  1.024489e-08  52.5
 
 [154 rows x 5 columns],
 'Buses':         Port Name  Year Measure       Density  Rank
 1       Madawaska  1996   Buses  9.220400e-08   1.0
 2          Morgan  1996   Buses  1.024489e-08   2.0
 3          Noonan  1996   Buses  0.000000e+00   3.0
 1

In [32]:
# Finding the top rank for each measure of transport for each year and diplaying density
# Filter top highest value for each measure
top_measure_dfs = {}
for measure, df in measure_dfs.items():
    top_measure_dfs[measure] = df[df['Rank'] <= 1]

# Combine the top 2 DataFrames into a single DataFrame
combined_top_df = pd.concat(top_measure_dfs.values()).sort_values(by=['Year', 'Measure', 'Rank']).reset_index(drop=True)

# Display the combined DataFrame
print(combined_top_df)

               Port Name  Year                  Measure       Density  Rank
0                  Alcan  1996           Bus Passengers  6.536239e-06   1.0
1              Madawaska  1996                    Buses  9.220400e-08   1.0
2               Eastport  1996        Personal Vehicles  3.847980e-05   1.0
3                  Sumas  1996                   Trains  8.913054e-07   1.0
4                Fortuna  1996                   Trucks  1.803101e-06   1.0
..                   ...   ...                      ...           ...   ...
164              Skagway  2023         Train Passengers  1.170376e-04   1.0
165  International Falls  2023                   Trains  3.384911e-06   1.0
166              Detroit  2023   Truck Containers Empty  3.041649e-04   1.0
167              Detroit  2023  Truck Containers Loaded  1.649492e-03   1.0
168              Detroit  2023                   Trucks  1.308694e-03   1.0

[169 rows x 5 columns]


In [33]:

# Calculate total value per season
total_value_per_season = ca.groupby('Season')['Value'].sum().reset_index()
total_value_per_season.rename(columns={'Value': 'Total Value'}, inplace=True)

# Merge with the original data to get the total value for each season
ca = pd.merge(ca, total_value_per_season, on='Season')

# Calculate density for each port and season
ca['Density'] = ca['Value'] / ca['Value'].sum()

# Group by Port Name, Season, and Measure to calculate average density
avg_density_per_measure_season = ca.groupby(['Port Name', 'Season', 'Measure'])['Density'].mean().reset_index()

# Rank the ports by density within each measure per season
avg_density_per_measure_season['Rank'] = avg_density_per_measure_season.groupby(['Season', 'Measure'])['Density'].rank(ascending=False)

# Sort the DataFrame by Season, Measure, and Rank
ranked_density_per_measure_season = avg_density_per_measure_season.sort_values(by=['Season', 'Measure', 'Rank']).reset_index(drop=True)

# Create a dictionary to hold DataFrames for each measure
measure_dfs_season = {}
for measure in ranked_density_per_measure_season['Measure'].unique():
    measure_dfs_season[measure] = ranked_density_per_measure_season[ranked_density_per_measure_season['Measure'] == measure]

# Filter the highest value for each measure in each season
highest_value_measure_dfs_season = {}
for measure, df_measure in measure_dfs_season.items():
    highest_value_measure_dfs_season[measure] = df_measure[df_measure['Rank'] == 1]

# Combine the highest value DataFrames into a single DataFrame
combined_highest_value_season_df = pd.concat(highest_value_measure_dfs_season.values()).sort_values(by=['Season', 'Measure', 'Rank']).reset_index(drop=True)

# Display the combined DataFrame
print(combined_highest_value_season_df)

                 Port Name  Season                      Measure   Density  \
0    Buffalo Niagara Falls    Fall               Bus Passengers  0.000126   
1                  Skagway    Fall                        Buses  0.000020   
2    Buffalo Niagara Falls    Fall                  Pedestrians  0.000062   
3    Buffalo Niagara Falls    Fall  Personal Vehicle Passengers  0.003551   
4    Buffalo Niagara Falls    Fall            Personal Vehicles  0.002378   
5               Port Huron    Fall        Rail Containers Empty  0.000156   
6      International Falls    Fall       Rail Containers Loaded  0.000495   
7                  Skagway    Fall             Train Passengers  0.000045   
8      International Falls    Fall                       Trains  0.000004   
9                  Detroit    Fall       Truck Containers Empty  0.000290   
10                 Detroit    Fall      Truck Containers Loaded  0.001304   
11                 Detroit    Fall                       Trucks  0.001264   

In [34]:
import pandas as pd
import folium

# Load the data
df = pd.read_csv('Border_Crossing_Entry_Data.csv')

# Add 'Year' and 'Season' columns
df['Date'] = pd.to_datetime(df['Date'], format='%b %Y')
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month

def month_to_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    elif month in [9, 10, 11]:
        return 'Fall'

df['Season'] = df['Month'].apply(month_to_season)

# Add a column for the border type
df['Border Type'] = df['Border'].apply(lambda x: 'US-Canada' if 'Canada' in x else 'US-Mexico')

# Calculate total value per season and border type
total_value_per_season_border = df.groupby(['Season', 'Year', 'Border Type'])['Value'].sum().reset_index()
total_value_per_season_border.rename(columns={'Value': 'Total Value'}, inplace=True)

# Merge with the original data to get the total value for each season and border type
df = pd.merge(df, total_value_per_season_border, on=['Season', 'Year', 'Border Type'])

# Calculate density for each port, season, and border type
df['Density'] = df['Value'] / df['Total Value']

# Group by Port Name, Season, Year, Measure, and Border Type to calculate average density
avg_density_per_measure_season_border = df.groupby(['Port Name', 'Season', 'Year', 'Measure', 'Border Type'])['Density'].mean().reset_index()

# Rank the ports by density within each measure, season, year, and border type
avg_density_per_measure_season_border['Rank'] = avg_density_per_measure_season_border.groupby(['Season', 'Year', 'Measure', 'Border Type'])['Density'].rank(ascending=False)

# Sort the DataFrame by Season, Year, Measure, Border Type, and Rank
ranked_density_per_measure_season_border = avg_density_per_measure_season_border.sort_values(by=['Season', 'Year', 'Measure', 'Border Type', 'Rank']).reset_index(drop=True)

# Filter the highest value for each measure in each season, year, and border type
highest_value_measure_dfs_season_border = ranked_density_per_measure_season_border[ranked_density_per_measure_season_border['Rank'] == 1]

# Add latitude, longitude, and point columns to the exportable CSV
highest_value_measure_dfs_season_border['Latitude'] = df['Latitude']  # Replace 'your_latitude_data' with your actual data
highest_value_measure_dfs_season_border['Longitude'] = df['Longitude']  # Replace 'your_longitude_data' with your actual data
highest_value_measure_dfs_season_border['Point'] = df['Point']

# Display the combined DataFrame
print(highest_value_measure_dfs_season_border)

# Save the DataFrame as a CSV file
highest_value_measure_dfs_season_border.to_csv('highest_rank_with_coordinates.csv', index=False)

# Example visualization with Folium
map_us = folium.Map(location=[37.0902, -95.7129], zoom_start=4)  # Center of the US
for index, row in highest_value_measure_dfs_season_border.iterrows():
    folium.Marker(
        [row['Latitude'], row['Longitude']],
        popup=f"Port: {row['Port Name']}, Density: {row['Density']}, Season: {row['Season']}, Year: {row['Year']}, Measure: {row['Measure']}, Border Type: {row['Border Type']}"
    ).add_to(map_us)

map_us.save('us_density_map.html')


<ipython-input-34-8c810038cbdf>:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  highest_value_measure_dfs_season_border['Latitude'] = df['Latitude']  # Replace 'your_latitude_data' with your actual data
<ipython-input-34-8c810038cbdf>:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  highest_value_measure_dfs_season_border['Longitude'] = df['Longitude']  # Replace 'your_longitude_data' with your actual data
<ipython-input-34-8c810038cbdf>:52: SettingWithCopyWarning: 
A value is trying to be set on a copy 

      Port Name  Season  Year                  Measure Border Type   Density  \
0        Noonan    Fall  1996                    Buses   US-Canada  0.000000   
1      Eastport    Fall  1996        Personal Vehicles   US-Canada  0.852087   
3        Scobey    Fall  1996                   Trains   US-Canada  0.000000   
4        Tecate    Fall  1996                   Trains   US-Mexico  0.000195   
5      Calexico    Fall  1996                   Trucks   US-Mexico  0.931556   
...         ...     ...   ...                      ...         ...       ...   
15693    Laredo  Winter  2023   Truck Containers Empty   US-Mexico  0.000881   
15714   Detroit  Winter  2023  Truck Containers Loaded   US-Canada  0.007338   
15785    Laredo  Winter  2023  Truck Containers Loaded   US-Mexico  0.002737   
15807   Detroit  Winter  2023                   Trucks   US-Canada  0.007558   
15882    Laredo  Winter  2023                   Trucks   US-Mexico  0.003489   

       Rank  Latitude  Longitude       

In [35]:
# Calculate average density
average_density = highest_value_measure_dfs_season_border['Density'].mean()
print("Average Density:", average_density)


Average Density: 0.14265431544101265


In [36]:
# Group by 'Port Name', 'Year', and 'Season' to aggregate the necessary information
grouped = df.groupby(['Port Name', 'Year', 'Season']).agg({
    'Value': 'sum',        # Sum of vehicles
    'Measure': lambda x: x.iloc[0],  # Vehicle type (assuming the first value, this might need adjustment)
    'Density': 'max',      # Highest density
    'State': 'first',      # Keep other relevant information
    'Port Code': 'first',
    'Border': 'first',
    'Latitude': 'first',
    'Longitude': 'first',
    'Point': 'first'
}).reset_index()

# Sort the dataframe to find the port with the most vehicles per year and season
sorted_grouped = grouped.sort_values(by=['Year', 'Season', 'Value'], ascending=[True, True, False]).reset_index(drop=True)

# Extract the port with the most vehicles, vehicle type, and the highest density for each year and season
result = sorted_grouped.loc[:, ['Port Name', 'Year', 'Season', 'Value', 'Measure', 'Density', 'State', 'Port Code', 'Border', 'Latitude', 'Longitude', 'Point']]

# Display the resulting dataframe
print(result)

# Save the result to a CSV file
result.to_csv('border_density.csv', index=False)

      Port Name  Year  Season    Value                      Measure   Density  \
0      Calexico  1996    Fall  14359.0                       Trucks  0.931556   
1      Eastport  1996    Fall   3756.0            Personal Vehicles  0.852087   
2          Roma  1996    Fall   1052.0                       Trucks  0.068250   
3        Turner  1996    Fall    652.0            Personal Vehicles  0.147913   
4        Tecate  1996    Fall      3.0                       Trains  0.000195   
...         ...   ...     ...      ...                          ...       ...   
3354     Opheim  2023  Winter    832.0  Personal Vehicle Passengers  0.000014   
3355    Ambrose  2023  Winter    531.0  Personal Vehicle Passengers  0.000011   
3356   Whitlash  2023  Winter    363.0            Personal Vehicles  0.000006   
3357     Hannah  2023  Winter    238.0       Truck Containers Empty  0.000005   
3358  Ketchikan  2023  Winter     19.0                  Pedestrians  0.000001   

             State  Port Co

In [37]:
result['Port Name'].value_counts()

Port Name
Roma             60
Morgan           57
Warroad          51
Calais           51
Laredo           49
                 ..
Portland          9
Friday Harbor     9
Anacortes         9
Bar Harbor        6
Noyes             4
Name: count, Length: 115, dtype: int64

In [38]:
result['Density'].max()

1.0

In [39]:
df

,Port Name,State,Port Code,Border,Date,Measure,Value,Latitude,Longitude,Point,Year,Month,Season,Border Type,Total Value,Density
0,Roma,Texas,2310,US-Mexico Border,2023-12-01,Buses,46.0,26.404,-99.019,POINT (-99.018981 26.403928),2023,12,Winter,US-Mexico,65601489.0,7.012036e-07
1,Del Rio,Texas,2302,US-Mexico Border,2023-12-01,Trucks,6552.0,29.327,-100.928,POINT (-100.927612 29.326784),2023,12,Winter,US-Mexico,65601489.0,9.987578e-05
2,Roma,Texas,2310,US-Mexico Border,2023-02-01,Trucks,3179.0,26.404,-99.019,POINT (-99.018981 26.403928),2023,2,Winter,US-Mexico,65601489.0,4.845927e-05
3,Rio Grande City,Texas,2307,US-Mexico Border,2023-12-01,Pedestrians,5697.0,26.366,-98.803,POINT (-98.802513 26.365761),2023,12,Winter,US-Mexico,65601489.0,8.684254e-05
4,Columbus,New Mexico,2406,US-Mexico Border,2023-02-01,Personal Vehicle Passengers,53243.0,31.784,-107.628,POINT (-107.627948 31.783742),2023,2,Winter,US-Mexico,65601489.0,8.116127e-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35681,Columbus,New Mexico,2406,US-Mexico Border,2017-12-01,Trucks,1305.0,31.784,-107.628,POINT (-107.627948 31.783742),2017,12,Winter,US-Mexico,5204004.0,2.507684e-04
35682,Nogales,Arizona,2604,US-Mexico Border,2017-12-01,Rail Containers Loaded,1983.0,31.333,-110.943,POINT (-110.942516 31.332839),2017,12,Winter,US-Mexico,5204004.0,3.810527e-04
35683,Tecate,California,2505,US-Mexico Border,2017-12-01,Personal Vehicle Passengers,173292.0,32.576,-116.626,POINT (-116.626216 32.576428),2017,12,Winter,US-Mexico,5204004.0,3.329974e-02
35684,Roma,Texas,2310,US-Mexico Border,2017-12-01,Bus Passengers,724.0,26.404,-99.019,POINT (-99.018981 26.403928),2017,12,Winter,US-Mexico,5204004.0,1.391236e-04


In [43]:
print(measure_traffic_pivot.columns)


Index(['Port Name', 'State', 'Port Code', 'Border', 'Date', 'Measure', 'Value',
       'Latitude', 'Longitude', 'Point', 'Year', 'Month', 'Season',
       'Border Type', 'Total Value', 'Density'],
      dtype='object')


In [45]:
# Ensure only numeric columns are normalized
numeric_columns = measure_traffic_pivot.select_dtypes(include='number').columns

# Pivot the DataFrame
measure_traffic_pivot = df.pivot_table(index=['Port Name', 'State'], columns='Measure', values='Value').reset_index()

# Normalize the measures
for measure in measure_traffic_pivot.columns[2:]:
    measure_traffic_pivot[measure] = measure_traffic_pivot[measure] / measure_traffic_pivot[measure].max()

# Assign weights
weights = {
    'Bus Passengers': 0.1,
    'Buses': 1,
    'Pedestrians': 0.1
}

# Calculate the composite score
measure_traffic_pivot['Composite Metric'] = sum(
    weights[measure] * measure_traffic_pivot[measure] for measure in weights
)

# Rank the ports based on the composite metric
measure_traffic_pivot['Rank'] = measure_traffic_pivot['Composite Metric'].rank(ascending=True)
measure_traffic_pivot = measure_traffic_pivot.sort_values('Rank')

# Display the resulting DataFrame
print(measure_traffic_pivot)

Measure    Port Name         State  Bus Passengers     Buses  Pedestrians  \
112         Whitlash       Montana        0.000000  0.000000     0.000002   
64         Nighthawk    Washington        0.000100  0.000000     0.000019   
61            Morgan       Montana        0.000139  0.000054     0.000001   
13       Bridgewater         Maine        0.000234  0.000251     0.000003   
107        Van Buren         Maine        0.000500  0.000251     0.000051   
..               ...           ...             ...       ...          ...   
103           Tecate    California             NaN  0.002636     0.112022   
104         Tornillo         Texas             NaN  0.000000     0.006179   
105      Trout River      New York        0.001051  0.000837          NaN   
111         Westhope  North Dakota             NaN  0.000377     0.000002   
115           Ysleta         Texas             NaN       NaN     0.185653   

Measure  Personal Vehicle Passengers  Personal Vehicles  \
112             

In [42]:
measure_traffic_pivot = pd.DataFrame(df)

# Ensure only numeric columns are normalized
numeric_columns = measure_traffic_pivot.select_dtypes(include='number').columns

# Step 1: Normalize the measures
for measure in numeric_columns:
    measure_traffic_pivot[measure] = measure_traffic_pivot[measure] / measure_traffic_pivot[measure].max()

# Step 2: Assign weights
weights = {
    'Bus Passengers': 0.1,
    'Buses': 1,
    'Pedestrians': 0.1,
    'Trains': 0.5,
    'Trucks': 0.5,
    'Personal Vehicle': 0.3
}

# Step 3: Calculate the composite score
measure_traffic_pivot['Composite Metric'] = sum(
    weights[measure] * measure_traffic_pivot[measure] for measure in weights
)

# Step 4: Rank the ports based on the composite metric
measure_traffic_pivot['Rank'] = measure_traffic_pivot['Composite Metric'].rank(ascending=True)
measure_traffic_pivot = measure_traffic_pivot.sort_values('Rank')

# Display the resulting DataFrame
print(measure_traffic_pivot)

KeyError: 'Bus Passengers'

In [46]:
# Pivot the DataFrame
measure_traffic_pivot = df.pivot_table(index=['Port Name', 'State'], columns='Measure', values='Value').reset_index()

# Normalize the measures
for measure in measure_traffic_pivot.columns[2:]:
    measure_traffic_pivot[measure] = measure_traffic_pivot[measure] / measure_traffic_pivot[measure].max()

# Assign weights
weights = {
    'Bus Passengers': 0.1,
    'Buses': 1,
    'Pedestrians': 0.1
}

# Calculate the composite score
measure_traffic_pivot['Composite Metric'] = sum(
    weights[measure] * measure_traffic_pivot[measure] for measure in weights
)

# Rank the ports based on the composite metric in ascending order to find the least busy ports
measure_traffic_pivot['Rank'] = measure_traffic_pivot['Composite Metric'].rank(ascending=True)
measure_traffic_pivot = measure_traffic_pivot.sort_values('Rank')

# Display the resulting DataFrame
print(measure_traffic_pivot)

Measure    Port Name         State  Bus Passengers     Buses  Pedestrians  \
112         Whitlash       Montana        0.000000  0.000000     0.000002   
64         Nighthawk    Washington        0.000100  0.000000     0.000019   
61            Morgan       Montana        0.000139  0.000054     0.000001   
13       Bridgewater         Maine        0.000234  0.000251     0.000003   
107        Van Buren         Maine        0.000500  0.000251     0.000051   
..               ...           ...             ...       ...          ...   
103           Tecate    California             NaN  0.002636     0.112022   
104         Tornillo         Texas             NaN  0.000000     0.006179   
105      Trout River      New York        0.001051  0.000837          NaN   
111         Westhope  North Dakota             NaN  0.000377     0.000002   
115           Ysleta         Texas             NaN       NaN     0.185653   

Measure  Personal Vehicle Passengers  Personal Vehicles  \
112             

In [79]:
# Preprocessing for ML model
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeRegressor

# Load the data
df = pd.read_csv('port_rank.csv')

# Encode the 'Season' column into numerical values
season_encoder = LabelEncoder()
df['Season'] = season_encoder.fit_transform(df['Season'])

# Encode the 'Measure' column into numerical values
measure_encoder = LabelEncoder()
df['Measure'] = measure_encoder.fit_transform(df['Measure'])

# Encode the port names into numerical labels
port_encoder = LabelEncoder()
df['Port Name'] = port_encoder.fit_transform(data['Port Name'])

# Split the data into features (X) and target (y)
X = df[['Season', 'Measure', 'Density', 'Rank']]
y = df[['Latitude', 'Longitude']]

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the Decision Tree model
dt_model = DecisionTreeRegressor(random_state=42)
dt_model.fit(X_train, y_train)

DecisionTreeRegressor(random_state=42)

In [61]:
from math import radians, cos, sin, asin, sqrt

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)
    """
    # Convert decimal degrees to radians
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    r = 6371  # Radius of earth in kilometers
    return c * r

# Create a new feature 'Distance' representing the distance from the user's location
user_lat, user_lon = 40.7128, -74.0060  # Example user location (New York City)
data['Distance'] = data.apply(lambda row: haversine(user_lon, user_lat, row['Longitude'], row['Latitude']), axis=1)

# Update the feature matrix X
X = data[['Season', 'Measure', 'Density', 'Rank', 'Distance']]

In [62]:
# Training decision tree and RFC
from sklearn.tree import DecisionTreeRegressor

# Train the Decision Tree model
dt_model = DecisionTreeRegressor(random_state=42)
dt_model.fit(X_train, y_train)

DecisionTreeRegressor(random_state=42)

In [63]:
# RF
from sklearn.ensemble import RandomForestRegressor

# Train the Random Forest model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [64]:
# Model Optimization

from sklearn.model_selection import RandomizedSearchCV
import multiprocessing

# Hyperparameters for Random Forest
rf_params = {
    'n_estimators': [1, 2, 10],
    'max_depth': [1, 2, 3, None],
    'min_samples_split': [2, 3, 4],
    'min_samples_leaf': [2, 3, 4]
}

# Number of CPUs available
n_cpus = multiprocessing.cpu_count()

# Random Search for Random Forest
rf_random_search = RandomizedSearchCV(
    RandomForestRegressor(random_state=42),
    param_distributions=rf_params,
    n_iter=50,  # Number of random combinations to try
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=n_cpus,  # Parallelize the search
    random_state=42
)

rf_random_search.fit(X_train, y_train)
best_rf_model = rf_random_search.best_estimator_
best_rf_model

RandomForestRegressor(min_samples_leaf=2, n_estimators=10, random_state=42)

In [65]:
# Hyperparameters for Random Forest
rf_params = {
    'n_estimators': [1, 2, 10],
    'max_depth': [1, 2, 3, None],
    'min_samples_split': [2, 3, 4],
    'min_samples_leaf': [2, 3, 4]
}

# Number of CPUs available
n_cpus = multiprocessing.cpu_count()

# Random Search for Random Forest
rf_random_search = RandomizedSearchCV(
    RandomForestRegressor(random_state=42),
    param_distributions=rf_params,
    n_iter=50,  # Number of random combinations to try
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=n_cpus,  # Parallelize the search
    random_state=42
)

# Fit the model
rf_random_search.fit(X_train, y_train)

# Extract the best model
best_rf_model = rf_random_search.best_estimator_

# Display the best parameters
print("Best Parameters:", rf_random_search.best_params_)

# Display the best score
print("Best Score (Negative Mean Squared Error):", rf_random_search.best_score_)

# Display feature importances
if hasattr(best_rf_model, 'feature_importances_'):
    print("Feature Importances:")
    for feature, importance in zip(X_train.columns, best_rf_model.feature_importances_):
        print(f"{feature}: {importance}")

# Display other model attributes
print("Number of Estimators:", best_rf_model.n_estimators)
print("Max Depth:", best_rf_model.max_depth)
print("Min Samples Split:", best_rf_model.min_samples_split)
print("Min Samples Leaf:", best_rf_model.min_samples_leaf)

Best Parameters: {'n_estimators': 10, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_depth': None}
Best Score (Negative Mean Squared Error): -5.4903080900313235
Feature Importances:
Season: 0.001139137696784394
Measure: 0.002390440501638198
Density: 0.14203669483688536
Rank: 0.8544337269646921
Number of Estimators: 10
Max Depth: None
Min Samples Split: 2
Min Samples Leaf: 2


In [66]:
# Model Optimizing
from sklearn.model_selection import GridSearchCV

# Hyperparameters for Decision Tree
dt_params = {
    'max_depth': [3, 5, 7, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Hyperparameters for Random Forest
rf_params = {
    'n_estimators': [1, 2, 10],
    'max_depth': [1, 2, 3, None],
    'min_samples_split': [2, 3, 4],
    'min_samples_leaf': [2, 3, 4]
}

# Grid Search for Decision Tree
dt_grid_search = GridSearchCV(DecisionTreeRegressor(random_state=42), dt_params, cv=5, scoring='neg_mean_squared_error')
dt_grid_search.fit(X_train, y_train)
best_dt_model = dt_grid_search.best_estimator_

# Grid Search for Random Forest
rf_grid_search = GridSearchCV(RandomForestRegressor(random_state=42), rf_params, cv=5, scoring='neg_mean_squared_error')
rf_grid_search.fit(X_train, y_train)
best_rf_model = rf_grid_search.best_estimator_

In [67]:
# Model Evaluation
from sklearn.metrics import mean_squared_error

# Evaluate the Decision Tree model
dt_predictions = best_dt_model.predict(X_test)
dt_mse = mean_squared_error(y_test, dt_predictions)
print(f'Decision Tree MSE: {dt_mse}')

# Evaluate the Random Forest model
rf_predictions = best_rf_model.predict(X_test)
rf_mse = mean_squared_error(y_test, rf_predictions)
print(f'Random Forest MSE: {rf_mse}')

Decision Tree MSE: 6.496680785175666
Random Forest MSE: 4.025567817361294


In [68]:
df

,Port Name,Year,Season,Value,Measure,Density,State,Port Code,Border,Latitude,Longitude,Point,Rank
0,34,1996,0,15840350,4,0.000333,Texas,2402,US-Mexico Border,31.764,-106.451,POINT (-106.451188 31.764363),5.0
1,34,1996,1,15884629,7,0.000338,Texas,2402,US-Mexico Border,31.764,-106.451,POINT (-106.451188 31.764363),5.0
2,34,1996,2,15823836,7,0.000319,Texas,2402,US-Mexico Border,31.764,-106.451,POINT (-106.451188 31.764363),5.0
3,34,1996,3,14693278,4,0.000335,Texas,2402,US-Mexico Border,31.764,-106.451,POINT (-106.451188 31.764363),5.0
4,34,1997,0,16011963,3,0.000318,Texas,2402,US-Mexico Border,31.764,-106.451,POINT (-106.451188 31.764363),5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12314,115,2023,1,3060276,2,0.000044,Texas,2401,US-Mexico Border,31.673,-106.335,POINT (-106.335449846028 31.6731261376859),31.0
12315,115,2023,2,3225843,10,0.000049,Texas,2401,US-Mexico Border,31.673,-106.335,POINT (-106.335449846028 31.6731261376859),31.0
12316,115,2023,3,3013136,11,0.000051,Texas,2401,US-Mexico Border,31.673,-106.335,POINT (-106.335449846028 31.6731261376859),31.0
12317,115,2024,1,1093148,4,0.000047,Texas,2401,US-Mexico Border,31.673,-106.335,POINT (-106.335449846028 31.6731261376859),31.0


In [74]:
# Calculating distance between two points
# Remove rows with NaN values
df = df.dropna(subset=['Latitude', 'Longitude', 'Density', 'Rank'])

def calculate_distances(df, point_a, point_b):
    df['Distance_to_B'] = df.apply(lambda row: geodesic((point_b['lat'], point_b['lon']), (row['Latitude'], row['Longitude'])).miles, axis=1)

    # Filter dataframe for points within a hundred-mile radius of point_b
    radius = 100  # 100 miles
    filtered_df = df[df['Distance_to_B'] <= radius]

    # Sort the filtered dataframe by distance to point_b and return the closest port location
    closest_port = filtered_df.loc[filtered_df['Distance_to_B'].idxmin()]

    return closest_port

In [92]:
import pandas as pd

# Convert the Series object to a DataFrame
df = pd.DataFrame(df)

# Check the type of df after conversion
print(type(df))  # This should now print pandas DataFrame

# Call the function only if df is a DataFrame
if isinstance(df, pd.DataFrame):
    df = calculate_composite_metric(df, weights)
else:
    print("Error: 'df' is not a pandas DataFrame.")


<class 'pandas.core.frame.DataFrame'>


In [93]:
import pandas as pd

# Read data from CSV file
input_file = 'Border_Crossing_Entry_Data.csv'
df = pd.read_csv(input_file)

# Convert 'Date' to datetime and extract 'Year'
df['Date'] = pd.to_datetime(df['Date'], format='%b %Y')
df['Year'] = df['Date'].dt.year

# Calculate total traffic for each port
total_traffic = df.groupby('Port Name')['Value'].sum().reset_index()
total_traffic.columns = ['Port Name', 'Total Traffic']

# Calculate traffic by measure
measure_traffic = df.groupby(['Port Name', 'Measure'])['Value'].sum().reset_index()
measure_traffic_pivot = measure_traffic.pivot(index='Port Name', columns='Measure', values='Value').fillna(0).reset_index()

# Normalize measures
for measure in measure_traffic_pivot.columns[1:]:
    measure_traffic_pivot[measure] = measure_traffic_pivot[measure] / measure_traffic_pivot[measure].max()

# Assign weights
weights = {
    'Bus Passengers': 0.1, 'Buses': 2, 'Pedestrians': 0.1,
    'Personal Vehicle Passengers': 0.1, 'Personal Vehicles': 1,
    'Rail Containers Empty': 0.2, 'Rail Containers Loaded': 1,
    'Train Passengers': 0.1, 'Trains': 2,
    'Truck Containers Empty': 0.2, 'Truck Containers Loaded': 1, 'Trucks': 2
}

# Calculate composite metric
measure_traffic_pivot['Composite Metric'] = sum(weights[measure] * measure_traffic_pivot[measure] for measure in weights)

# Merge total traffic
port_metrics = measure_traffic_pivot.merge(total_traffic, on='Port Name', how='left')

# Function to calculate composite score
def calculate_composite_metric(df, weights):
    if not isinstance(df, pd.DataFrame):
        raise TypeError("Input 'df' must be a pandas DataFrame.")

    # Ensure column names in df match the keys in weights
    common_columns = set(df.columns).intersection(weights.keys())

    # Calculate composite metric using common columns
    df['Composite Metric'] = sum(weights[measure] * df[measure] for measure in common_columns)
    return df


# Calculate composite metric using the function
port_metrics = calculate_composite_metric(port_metrics, weights)

port_metrics


,Port Name,Bus Passengers,Buses,Pedestrians,Personal Vehicle Passengers,Personal Vehicles,Rail Containers Empty,Rail Containers Loaded,Train Passengers,Trains,Truck Containers Empty,Truck Containers Loaded,Trucks,Composite Metric,Total Traffic
0,Alcan,0.010365,0.006094,2.840729e-05,0.003841,0.003366,0.0,0.000000e+00,0.0,0.000000,2.709847e-03,3.869077e-03,0.003374,0.028138,4787856
1,Alexandria Bay,0.061952,0.024279,1.161099e-04,0.053447,0.042631,0.0,0.000000e+00,0.0,0.000000,2.811263e-02,1.448622e-01,0.119433,0.492091,69431055
2,Algonac,0.000000,0.000000,0.000000e+00,0.000192,0.000448,0.0,0.000000e+00,0.0,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000467,317499
3,Ambrose,0.000010,0.000006,1.428459e-08,0.000196,0.000174,0.0,0.000000e+00,0.0,0.000000,9.507787e-05,1.604697e-05,0.000082,0.000405,219310
4,Anacortes,0.000308,0.000172,4.798908e-04,0.001501,0.001196,0.0,0.000000e+00,0.0,0.000000,0.000000e+00,0.000000e+00,0.000105,0.001978,1690849
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,Whitetail,0.000018,0.000007,1.666535e-07,0.000144,0.000128,0.0,0.000000e+00,0.0,0.000000,2.517519e-05,1.854190e-05,0.000045,0.000272,160092
112,Whitlash,0.000000,0.000000,4.428223e-07,0.000055,0.000045,0.0,0.000000e+00,0.0,0.000000,1.159081e-04,1.415876e-04,0.000151,0.000518,73287
113,Wildhorse,0.000119,0.000053,7.142294e-08,0.000787,0.000664,0.0,0.000000e+00,0.0,0.000000,1.704117e-04,9.531785e-04,0.000706,0.003260,916403
114,Willow Creek,0.000007,0.000003,8.046985e-07,0.000158,0.000146,0.0,0.000000e+00,0.0,0.000000,1.277928e-07,2.268123e-07,0.000004,0.000177,174327


In [94]:
# Check the type of df before calling the function
print(type(df))  # This will print the type of df, make sure it's pandas DataFrame

# Call the function only if df is a DataFrame
if isinstance(df, pd.DataFrame):
    df = calculate_composite_metric(df, weights)
else:
    print("Error: 'df' is not a pandas DataFrame.")


<class 'pandas.core.frame.DataFrame'>


In [98]:
import pandas as pd

# Convert the Series object to a DataFrame
df = pd.DataFrame(df)

# Check if df is a DataFrame before calling the function
if isinstance(df, pd.DataFrame):
    df = calculate_composite_metric(df, weights)
else:
    raise TypeError("Input 'df' must be a pandas DataFrame.")

# Continue with the rest of your code


In [100]:
df

,429,Composite Metric
Port Name,Ogdensburg,0
State,New York,0
Port Code,701,0
Border,US-Canada Border,0
Date,2006-04-01 00:00:00,0
Measure,Buses,0
Value,14,0
Latitude,44.733,0
Longitude,-75.458,0
Point,POINT (-75.4577501759 44.7330898624),0


In [99]:
import pandas as pd
from geopy.distance import geodesic
# Finding the optimal point
def find_optimal_port(df):
    optimal_port = df.loc[df['Composite_Score'].idxmax()]
    return optimal_port

# Example user input points
point_a = {'lat': 40.7128, 'lon': -74.0060}  # Example coordinates for point A
point_b = {'lat': 45.4215, 'lon': -75.6981}  # Example coordinates for point B



# Calculate distances
df = calculate_distances(df, point_a, point_b)

# Calculate composite score
df = calculate_composite_metric(df, weights)

# Find the optimal port
optimal_port = find_optimal_port(df)

print(optimal_port)


KeyError: 'Latitude'